# RooTrip 추천 시스템 v1

In [310]:
import pandas as pd
import numpy as np
import os

os.chdir('/Users/woojjam/Desktop/study/machine-learning/Projects/rootrip-recommendation')

df1 = pd.read_csv('rootrip-board.csv')
print(df1)

   id                                    Hashtag        like      title
0   1              [{"tag":"어트랙션"},{"tag":"여행"}]        1234    어트랙션 여행
1   2                [{"tag":"바다"},{"tag":"여행"}]      124124      바다 여행
2   3    [{"tag":"산"},{"tag":"바다"},{"tag":"관광"}]         211    산 바다 여행
3   4  [{"tag":"부산"},{"tag":"바다"},{"tag":"해운대"}]  1232145123  부산 바다 해운대


In [311]:
df1.head()

,id,Hashtag,like,title
0,1,"[{""tag"":""어트랙션""},{""tag"":""여행""}]",1234,어트랙션 여행
1,2,"[{""tag"":""바다""},{""tag"":""여행""}]",124124,바다 여행
2,3,"[{""tag"":""산""},{""tag"":""바다""},{""tag"":""관광""}]",211,산 바다 여행
3,4,"[{""tag"":""부산""},{""tag"":""바다""},{""tag"":""해운대""}]",1232145123,부산 바다 해운대


In [194]:
df1.shape

(4, 4)

In [312]:
from ast import literal_eval
df1['Hashtag'] = df1['Hashtag'].apply(literal_eval)

In [313]:
df1

,id,Hashtag,like,title
0,1,"[{'tag': '어트랙션'}, {'tag': '여행'}]",1234,어트랙션 여행
1,2,"[{'tag': '바다'}, {'tag': '여행'}]",124124,바다 여행
2,3,"[{'tag': '산'}, {'tag': '바다'}, {'tag': '관광'}]",211,산 바다 여행
3,4,"[{'tag': '부산'}, {'tag': '바다'}, {'tag': '해운대'}]",1232145123,부산 바다 해운대


In [314]:
def get_value(x):
    if isinstance(x, list):
        hashtag = [i['tag'] for i in x]
        return hashtag
    return []

In [315]:
df1['Hashtag'] = df1['Hashtag'].apply(get_value)


In [316]:
df1

,id,Hashtag,like,title
0,1,"[어트랙션, 여행]",1234,어트랙션 여행
1,2,"[바다, 여행]",124124,바다 여행
2,3,"[산, 바다, 관광]",211,산 바다 여행
3,4,"[부산, 바다, 해운대]",1232145123,부산 바다 해운대


In [318]:
def create_soup(x):
    hashtag = ' '.join(x['Hashtag']) if x['Hashtag'] else ''
    return hashtag

In [326]:
df1['soup'] = df1.apply(create_soup, axis=1)
df1['Hashtag'] = df1.apply(create_soup, axis=1)
df1['soup'], df1['Hashtag']

(0      어트랙션 여행
 1        바다 여행
 2      산 바다 관광
 3    부산 바다 해운대
 Name: soup, dtype: object,
 0      어트랙션 여행
 1        바다 여행
 2      산 바다 관광
 3    부산 바다 해운대
 Name: Hashtag, dtype: object)

In [328]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix = count.fit_transform(df1['soup'])
count_matrix

<4x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [329]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

array([[1.        , 0.5       , 0.        , 0.        ],
       [0.5       , 1.        , 0.5       , 0.40824829],
       [0.        , 0.5       , 1.        , 0.40824829],
       [0.        , 0.40824829, 0.40824829, 1.        ]])

In [330]:
df1.reset_index()
indices = pd.Series(df1.index, index=df1['Hashtag']).drop_duplicates()
print(indices)

Hashtag
어트랙션 여행      0
바다 여행        1
산 바다 관광      2
부산 바다 해운대    3
dtype: int64


In [331]:
user_hashtag = '부산 바다 해운대'

In [332]:
# 영화의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개으 영화 목록 반환
def get_recommendations(user_hashtag, cosine_sim=cosine_sim):
    # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
    idx = indices[user_hashtag]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))

    
    # 코사인 유사도 매트릭스 (cosine_sim)에서 idx에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
    # sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 코사인 유사도 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
    sim_scores = sim_scores[:3]
    print(sim_scores)
    # 추천 영화 목록 10개의 인덱스 정보 추출
    hashtag_indices = [i[0] for i in sim_scores]
    
    # 인덱스 정보를 통해 영화 제목 추출
    return df1['Hashtag'].iloc[hashtag_indices]


In [333]:
recommendation = get_recommendations(user_hashtag, cosine_sim=cosine_sim2)
print(recommendation)

3
[(3, 1.0000000000000002), (1, 0.408248290463863), (2, 0.408248290463863)]
3    부산 바다 해운대
1        바다 여행
2      산 바다 관광
Name: Hashtag, dtype: object
